# Чат-бот с искусственным интелектом на Python

По мотивам трехдневного одноименного интенсива от Skillbox.

Работа проводилась в программной среде Goggle Colab. Понадобились следующие библиотеки: random, json, sklearn, telegram.


In [31]:
# Установка в Google Colab библиотеки для работы с telegram ботом
# !pip install python-telegram-bot

In [32]:
# Подключение необходимых библиотек

import random
import json

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [33]:
# Загрузка данных для ML 
# и разделение их на тренировочную и тестовую части

with open('/BOT_CONFIG.json') as f:
  BOT_CONFIG = json.load(f)

X, y = list(), list()
for intent in BOT_CONFIG['intents']:
  for example in BOT_CONFIG['intents'][intent]['examples']:
    X.append(example)
    y.append(intent)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
# Обучение модели

vectorizer = CountVectorizer(ngram_range=(1, 3), analyzer='char_wb')
X_train_vectorized = vectorizer.fit_transform(X_train)
x_test_vectorized = vectorizer.transform(X_test)

model = RandomForestClassifier()
model.fit(X_train_vectorized, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [35]:
# Оценка работы модели
print(f'Точность модели на тренировочных данных: {100 * model.score(X_train_vectorized, y_train):0.2f}%')
print(f'Точность модели на тестовых данных: {100 * model.score(x_test_vectorized, y_test):0.2f}%')


Точность модели на тренировочных данных: 91.03%
Точность модели на тестовых данных: 19.38%


In [36]:
# Функции для работы бота
def get_intent_by_model(text):
  return model.predict(vectorizer.transform([text]))[0]

def bot(text):
  intent = get_intent_by_model(text)
  return random.choice(BOT_CONFIG['intents'][intent]['responses'])

def testing():
  input_text = ''
  while input_text != 'exit':
    input_text = input()
    response = bot(input_text)
    print(response)

In [37]:
# Функции для запуска бота в telegram
# Сам бот создается с помощью телеграм бота @BotFather

def start(update: Update, context: CallbackContext) -> None:
    # Приветствие при старте бота
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )

def answer(update: Update, context: CallbackContext) -> None:
    # Ответ на сообщение
    answer = bot(update.message.text)
    update.message.reply_text(answer)


def start_the_bot() -> None:
  # Setting up the bot
  # Token можно взять из @BotFather
  bot = Updater("Token")
  dispatcher = bot.dispatcher
  dispatcher.add_handler(CommandHandler("start", start))
  dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, answer))

  # Start the Bot
  bot.start_polling()
  bot.idle()

In [ ]:
# Запуск бота или функции для тестов

# start_the_bot()
testing()

### Итоги

Бот работает. Конечно, не так точно, как хотелось бы, но это ведь только обучающий пример.

Чтобы *улучшить* работу бота можно было бы:

* Использовать более эффектиный векторайзер
* Использовать иные классификаторы для распознания интента
* Оптимизировать гиперпараметры модели и векторайзера
* Собрать больше данных для обучения